In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil import relativedelta
from utility import get_all_teams

In [2]:
naz_under21=pd.read_excel("Punti_Nazionale_Under21.xlsx")
naz_under21=naz_under21.iloc[::-1]
naz_under21=naz_under21[10:-1]
naz_under21=naz_under21[["Anno","Punti per partita","Europeo"]]
naz_under21["Points_naz_u21"]=naz_under21["Punti per partita"]+naz_under21["Europeo"]

naz_under21.drop(["Europeo","Punti per partita"],axis=1,inplace=True)
naz_under21=naz_under21.set_index('Anno')
naz_under21.index.name = None

In [3]:
italy_ranking = pd.DataFrame()
for i in range(1992, 2023):
    df_temp = pd.read_excel('Ranking_Italy.xlsx', sheet_name=str(i))
    mean= df_temp['Total Points'].mean()
    df_temp = df_temp[df_temp['Team'] == 'Italy']
    df_temp['Year'] = i
    df_temp['Points'] = df_temp['Total Points']/mean
    italy_ranking = pd.concat([italy_ranking, df_temp], ignore_index=True)
italy_ranking = italy_ranking[['Year', 'RK','Total Points','Points']]
italy_ranking['Year'] = italy_ranking['Year'].astype(int)
italy_ranking['RK'] = italy_ranking['RK'].astype(int)
italy_ranking['Points'] = (italy_ranking['Points']+(1/italy_ranking['RK'])).astype(float)
#italy_ranking["Points"][14:17]=italy_ranking["Points"][14:17]-0.55
#italy_ranking["Points"][15:16]=italy_ranking["Points"][15:16]-0.2
italy_ranking=italy_ranking[['Year','Points']]
italy_ranking=italy_ranking.set_index('Year')
italy_ranking.index.name = None
italy_ranking.rename(columns={'Points':'Fifa_points'}, inplace=True)
italy_ranking.head()

,Fifa_points
1992,1.856497
1993,1.623797
1994,1.823655
1995,1.367495
1996,1.404581


In [ ]:
#get_table_result()
#get_transfermarkt(2022,56)

In [4]:
valori_01_12 =pd.read_excel("ITACPI.xls")
valori_01_12 = valori_01_12 [valori_01_12['date'].dt.month == 12]
valori_01_12.set_index(valori_01_12['date'].dt.year,inplace=True)
valori_01_12.drop(["date"],axis=1,inplace=True)
valori_01_12.index.name = None
valori_01_12=valori_01_12[7:]
valori_01_12[23:40]=valori_01_12[23:40]-18

In [5]:
dataframe=pd.DataFrame()
for i in range(1993,2022):
    data=pd.read_csv(f"data/{i}/serie-a.csv")
    #show the possible value on column movement
    data=data[data['movement']=='out']
    data=data[data['age']<=23]
    data["fee"].sum()
    #add the data to the dataframe on the column fee
    dataframe.loc[i,"fee"]=data["fee"].sum()
dataframe[-1:]=dataframe[-1:]+22262500
dataframe.loc[2022,"fee"]=58262500

In [6]:
dataframe=dataframe/1000000
dataframe=dataframe.join(valori_01_12,how="inner")      
dataframe["fee"]=dataframe["fee"]/dataframe["ITACPI"]*100
dataframe["revenue"]=np.log(dataframe["fee"])
dataframe.drop(["fee","ITACPI"],axis=1,inplace=True)

In [7]:
# Calcola i minuti giocati dai giocatori under 21 per ogni anno, stabilendo come under 21 i giocatori nati dopo il 1 Agosto 
all_teams=get_all_teams()
mesi = {
    "gen": 1, "feb": 2, "mar": 3, "apr": 4, "mag": 5, "giu": 6, "lug": 7, "ago": 8, "set": 9, "ott": 10, "nov": 11, "dic": 12
}
mins_played=pd.DataFrame(columns=["Minutes_played_under21","Minutes_played_italians","Minutes_played_foreigners","Minutes_played_oriundi"])
current_year = 2022
year_to_subtract = 56

for i in range(year_to_subtract):
    # Calculate the year to use in the URL
    year_url = f"{current_year - i}-{current_year + 1 - i}"
    players_tm=pd.read_excel(f"transfermarkt\SerieA{year_url}-Under21.xlsx")
    for j in range(len(players_tm)):
        parti = players_tm.loc[j,"Nato"].split('/')
        giorno_nasc = int(parti[0])
        mese_nasc = mesi[parti[1].lower()]  # Mappa il mese in minuscolo
        anno_nasc = int(parti[2])
        data_nasc = datetime(anno_nasc, mese_nasc, giorno_nasc)
        players_tm.loc[j,"Data di nascita"]=data_nasc
        data_inizio=datetime(current_year-i, 1, 1)
        anni=relativedelta.relativedelta(data_inizio, data_nasc).years
        players_tm.loc[j,"Anni"]=anni
    
    under=players_tm[players_tm["Anni"]<21]
    under_italia=under[under["Nazione"]=="Italia"]
    under_stranieri= under[~under["Nazione"].str.contains(r'\b^Italia$\b|^Italia/\b', case=False, na=False)]
    under_oriundi = under[under["Nazione"].str.contains(r'^Italia/\b', case=False, na=False)]
    mins_played.loc[year_url,"Minutes_played_italians"]=under_italia["Minuti giocati"].sum()
    mins_played.loc[year_url,"Minutes_played_under21"]=under["Minuti giocati"].sum()
    mins_played.loc[year_url,"Minutes_played_foreigners"]=under_stranieri["Minuti giocati"].sum()
    mins_played.loc[year_url,"Minutes_played_oriundi"]=under_oriundi["Minuti giocati"].sum()
   
    # count the minutes played for each team
    team_minutes=players_tm[["Squadra","Minuti giocati"]].groupby("Squadra").sum().sort_values(by="Minuti giocati",ascending=False).transpose()
    #set the index of the dataframe
    team_minutes.columns.name = None
    team_minutes=team_minutes.reset_index(drop=True)
    team_minutes["Anno"]=year_url
    team_minutes=team_minutes.set_index('Anno')
    team_minutes.index.name = None
    all_teams.loc[year_url] = team_minutes.loc[year_url]

In [8]:
"""normalize Minuti giocati to total minutes played in the league and create another column
(from 2022-2023 to 2004-2005 divided for 3420 minutes,
from 2003-2004 to 1988-1989 divided for 3060 minutes
from 1987-1988 to 1968-1969 divided for 2700 minutes)"""

mins_played=mins_played.iloc[::-1]
#1967, cioè -56
mins_played[0:21]=mins_played[0:21]/(90*11*30*20)*100
mins_played[21:37]=mins_played[21:37]/(90*11*34*20)*100
mins_played[37:]=mins_played[37:]/(90*11*38*20)*100

mins_played["Minutes_played_under21"]=mins_played["Minutes_played_under21"].astype(float)
mins_played["Minutes_played_italians"]=mins_played["Minutes_played_italians"].astype(float)
mins_played["Minutes_played_foreigners"]=mins_played["Minutes_played_foreigners"].astype(float)
mins_played["Minutes_played_oriundi"]=mins_played["Minutes_played_oriundi"].astype(float)


In [9]:
#!!!!!!join the dataframe mins_played with uefa_ranking and italy_ranking!!!!!!!!
mins_played.index=mins_played.index.str[:4].astype(int)
mins_played=mins_played.join(italy_ranking,how='outer')
mins_played=mins_played.join(naz_under21,how='outer')
mins_played=mins_played.join(dataframe,how='outer')


In [11]:
mins_played.tail()

,Minutes_played_under21,Minutes_played_italians,Minutes_played_foreigners,Minutes_played_oriundi,Fifa_points,Points_naz_u21,revenue
2018,8.221823,2.461855,5.552100,0.207868,1.064686,1.350,5.489717
2019,6.539474,2.515949,3.920654,0.102871,1.133279,2.555,6.272201
2020,6.049575,1.714779,4.045853,0.288942,1.159438,3.000,5.480558
2021,5.818846,1.762759,3.514886,0.541201,1.386975,2.475,5.337017
2022,6.184476,2.403907,3.013557,0.767012,1.312050,1.750,3.891005
